In [ ]:
%load_ext autoreload
%autoreload 1

%aimport sheet
%aimport results_comparer
%aimport plot
%aimport settings_handler



import cv2 as cv
import numpy as np
import matplotlib.pyplot as plt
import ipywidgets as widgets
from IPython.display import display
from IPython.display import FileLink
import os
from IPython.display import clear_output, display


In [ ]:
sheet.initial_display()

In [ ]:
results_comparer.display_controls()

In [ ]:
plot.display_image_settings()
plot.show_final(True, 3)    

In [ ]:
settings_handler.display_settings()